In [1]:
from qutip import *
from qutip import gates
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


In [2]:
q =Qobj([[1,0],[1,2]])


print(q)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[1. 0.]
 [1. 2.]]


In [3]:
qubit_0 = basis(2, 0)  # |0⟩ state
qubit_1 = basis(2, 0)  # |0⟩ state
initial_state = tensor(qubit_0, qubit_1)  # Tensor product of the qubits

# Define the Hadamard gate (H) on qubit 1
H = gates.hadamard_transform(1)

# Define the CNOT gate
CNOT = gates.cnot()

# Apply the Hadamard gate to the first qubit (tensor identity on the second qubit)
H_on_first_qubit = tensor(H, qeye(2))  # H on qubit 1, identity on qubit 2
state_after_H = H_on_first_qubit * initial_state  # Apply Hadamard gate

# Apply the CNOT gate
state_after_CNOT = CNOT * state_after_H  # Apply CNOT

# Print the final quantum state
state_after_CNOT


Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

In [4]:
tensor(state_after_CNOT, state_after_CNOT.dag()).eigenenergies()


array([0., 0., 0., 1.])

In [5]:

def look_o(theta, base):

    qubit_0 = basis(2, 0)  # |0⟩ state
    qubit_1 = basis(2, 0)  # |0⟩ state
    state = tensor(qubit_0, qubit_1)  # Tensor product of the qubits
    H=gates.hadamard_transform()

    state  = tensor(H, qeye(2))*state
    CNOT = gates.cnot()
    state = CNOT * state
    rx = gates.rx(theta)
    state  = tensor(rx, rx)*state
    state = CNOT * state
    return state[base,0]

def look_S(theta):

    qubit_0 = basis(2, 0)  # |0⟩ state
    qubit_1 = basis(2, 0)  # |0⟩ state
    state = tensor(qubit_0, qubit_1)  # Tensor product of the qubits
    H=gates.hadamard_transform()

    state  = tensor(H, qeye(2))*state
    CNOT = gates.cnot()
    state = CNOT * state
    rx = gates.rx(theta)
    state  = tensor(rx, rx)*state
    state = CNOT * state
    return state
look_S(np.pi)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[-0.70710678]
 [ 0.        ]
 [-0.70710678]
 [ 0.        ]]

In [6]:
def sp_qutip(theta,n_qubit):
    qb=[basis(2, 0) for _ in range(n_qubit)]
    state = tensor(qb)  # Tensor product of the qubits
    H=gates.hadamard_transform()
    hi=[H] + [qeye(2)]*(n_qubit-1)
    state  = tensor(hi)*state
    CNOT = gates.cnot()
    for i in range(n_qubit-1):

        cnoti =[ qeye(2)]*(i) + [CNOT]
        cnoti += [qeye(2)]*(n_qubit-2-i)
        state = tensor(cnoti) * state
        
    rx = gates.rx(theta)
    rxx= [rx]*n_qubit
    state  = tensor(rxx)*state
    for i in range(n_qubit-1):
        cnoti =[ qeye(2)]*(i) + [CNOT]
        cnoti += [qeye(2)]*(n_qubit-2-i)
        state = tensor(cnoti) * state
    return state
sp_qutip(np.pi,3)

Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[ 1.62340758e-49+7.07106781e-01j]
 [-4.32978028e-17-2.65122578e-33j]
 [-4.32978028e-17-2.65122578e-33j]
 [-4.32978028e-17-2.65122578e-33j]
 [-4.32978028e-17-2.65122578e-33j]
 [ 1.62340758e-49+7.07106781e-01j]
 [-4.32978028e-17-2.65122578e-33j]
 [-4.32978028e-17-2.65122578e-33j]]

In [7]:
import matplotlib.pyplot as plt
import numpy as np

def plot_real_imag_dual_axis(complex_list, title="Real and Imaginary Parts Evolution"):
    """
    Plots the real part on one y-axis and the imaginary part on another y-axis.

    Parameters:
    - complex_list: list of complex numbers
    - title: title of the plot
    """
    # Extract real and imaginary parts
    real_parts = [z.real for z in complex_list]
    imaginary_parts = [z.imag for z in complex_list]
    
    # Create an index for the x-axis (time or index)
    x = np.linspace(0,np.pi*2, 1000)

    # Create the figure and the first axis (for the real part)
    fig, ax1 = plt.subplots(figsize=(4, 3))

    # Plot the real part on ax1
    ax1.plot(x, real_parts, 'b-', label='Real Part')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('Real Part', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Create a second y-axis (for the imaginary part) that shares the same x-axis
    ax2 = ax1.twinx()  
    ax2.plot(x, imaginary_parts, 'r-', label='Imaginary Part')
    ax2.set_ylabel('Imaginary Part', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Add title
    plt.title(title)

    # Add grid to both axes
    ax1.grid(True)

    # Show the plot
    fig.tight_layout()  # Adjust the layout so that labels don't overlap
    plt.show



#Evolution of the 4 basis 
if False:
    for base in range(4):
        a =[]
        for theta in np.linspace(0,np.pi*2, 1000):
            a.append(look_o(theta,base))
        plot_real_imag_dual_axis(a)
    


In [8]:
a =[]
qb_input_state=4
qb_trash_state=2
for theta in np.linspace(0,np.pi*2, 1000):
    a.append(sp_qutip(theta,qb_trash_state))


In [22]:
b =np.sum([tensor(c, c.dag()) for c in a])/len(a)

In [9]:
np.sum(b.eigenenergies()[qb_trash_state-qb_input_state:])

1.0